# Notebook 2 : Pandas avancé

In [130]:
import pandas as pd

## Tennis

Nous considérons les données des résultats des matchs de tennis masculin des tournois de Roland Garros et Wimbledon en 2013. La liste des variables et leur signification se trouvent sur [cette page](https://archive.ics.uci.edu/dataset/300/tennis+major+tournament+match+statistics) dans la section *Additional Variable Information*.

1. Commencer par charger le jeu de données relatif au tournoi de Roland Garros dans un dataframe `rg` à partir du fichier `rolandgarros2013.csv`.

In [131]:
rg2013 = pd.read_csv("C:/Users/cepe-s3-02/Desktop/David/Bloc 2/notebooks/data/rolandgarros2013.csv")

wb2013 = pd.read_csv("C:/Users/cepe-s3-02/Desktop/David/Bloc 2/notebooks/data/wimbledon2013.csv")

print(rg2013)

                 Player1                  Player2  Round  Result  FNL.1  \
0    Pablo Carreno-Busta            Roger Federer      1       0      0   
1       Somdev Devvarman  Daniel Munoz-De La Nava      1       1      3   
2           Tobias Kamke            Paolo Lorenzi      1       1      3   
3       Julien Benneteau        Ricardas Berankis      1       1      3   
4            Lukas Lacko              Sam Querrey      1       0      0   
..                   ...                      ...    ...     ...    ...   
120         Rafael Nadal       Stanislas Wawrinka      5       1      3   
121       Novak Djokovic               Tommy Haas      5       1      3   
122         David Ferrer       Jo-Wilfried Tsonga      6       1      3   
123       Novak Djokovic             Rafael Nadal      6       0      2   
124         Rafael Nadal             David Ferrer      7       1      3   

     FNL.2  FSP.1  FSW.1  SSP.1  SSW.1  ...  BPC.2  BPW.2  NPA.2  NPW.2  \
0        3     62     27

2. Afficher les noms des demi-finalistes.

In [132]:
rg2013Final = rg2013[rg2013["Round"] == 6]

print(rg2013Final.Player1 + " vs " + rg2013Final.Player2)


122    David Ferrer vs Jo-Wilfried Tsonga
123        Novak Djokovic vs Rafael Nadal
dtype: object


3. Calculer le nombre moyen d'aces par match dans le tournoi.

In [133]:
meanAces = (rg2013["ACE.1"] + rg2013["ACE.2"]).mean()

print(meanAces)

12.688


4. Combien y a-t-il eu d'aces par match en moyenne à chaque niveau du tournoi ?

In [134]:
rg2013["ACE"] = rg2013["ACE.1"] + rg2013["ACE.2"]

meanAcesByRound = rg2013.groupby("Round")["ACE"].mean()

print(meanAcesByRound)


Round
1    13.476190
2    13.193548
3    12.562500
4     9.125000
5     7.000000
6    10.000000
7     6.000000
Name: ACE, dtype: float64


5. Filtrer les matchs pour lesquels au moins une des variables `DBF.1` et `DBF.2` est manquante.

In [135]:
rg2013["MATCH"] = rg2013["Player1"] + " vs " + rg2013["Player2"]

# print(rg2013["MATCH"])

missing_dbf = rg2013[rg2013["DBF.1"].isna() | rg2013["DBF.2"].isna()]

print(missing_dbf[["MATCH", "DBF.1", "DBF.2"]])

                                MATCH  DBF.1  DBF.2
56      Simone Bolelli vs Yen-Hsun Lu    NaN    NaN
63  Somdev Devvarman vs Roger Federer    NaN    NaN


6. Remplacer les valeurs manquantes de `DBF.1` par zéro avec la méthode `loc`.

In [136]:
rg2013.loc[rg2013["DBF.1"].isna(), "DBF.1"] = 0

7. Remplacer les valeurs manquantes de `DBF.2` par zéro avec la méthode `fillna`.

In [137]:
rg2013["DBF.2"] = rg2013["DBF.2"].fillna(0)

8. Extraire la liste des participants à partir des colonnes `Player1` et `Player2`. Une façon de faire consiste à utiliser `concat` et la méthode `drop_duplicates` pour obtenir le résultat sous la forme d'une série et de la convertir en dataframe avec la méthode `to_frame`.

In [138]:
players = pd.concat([rg2013["Player1"], rg2013["Player2"]])

players.drop_duplicates(inplace=True)

print(players)

0     Pablo Carreno-Busta
1        Somdev Devvarman
2            Tobias Kamke
3        Julien Benneteau
4             Lukas Lacko
             ...         
56            Yen-Hsun Lu
59        Grigor Dimitrov
61            Guido Pella
62           David Goffin
87       Janko Tipsarevic
Length: 127, dtype: object


9. Écrire une fonction `n_match` qui prend une chaîne de caractères `joueur` en entrée et retourne le nombre de matchs disputés par le joueur.

In [139]:
def n_match(matchs, joueur):

    # Compter les occurrences du joueur dans Player1 et Player2
    count_player1 = matchs[matchs["Player1"] == joueur]["Player1"].count()
    count_player2 = matchs[matchs["Player2"] == joueur]["Player2"].count()

    # Calculer le total des matchs joués
    n_match = count_player1 + count_player2
    
    return n_match

n_match(rg2013, "David Ferrer")


7

10. Utiliser les deux question précédentes et la méthode `apply` pour compter le nombre de matchs que chaque participant a disputé et ordonner le résultat par ordre décroissant.

In [140]:
playersDf = players.to_frame(name="Joueur")

playersDf["nMatchs"] = 0

for joueur in players:
    nbMatchParJoueur = n_match(rg2013, joueur)
    #print(joueur + " : " + str(nbMatchParJoueur))
    playersDf.loc[playersDf["Joueur"] == joueur, "nMatchs"] = nbMatchParJoueur

print(playersDf.sort_values("nMatchs", ascending=False))

#playersDf["nMatchs"] = playersDf.Joueur.apply(n_match)
#print(players)

                  Joueur  nMatchs
47          Rafael Nadal        7
122         David Ferrer        7
62        Novak Djokovic        6
15    Jo-Wilfried Tsonga        6
113        Tommy Robredo        5
..                   ...      ...
5            Denis Kudla        1
19     Marcel Granollers        1
51        Carlos Berlocq        1
10           James Blake        1
0    Pablo Carreno-Busta        1

[127 rows x 2 columns]


11. Charger maintenant le jeu de données relatif au tournoi de Wimbledon dans un dataframe `wb` à partir du fichier `wimbledon2013.csv`.

In [141]:
wb2013 = pd.read_csv("C:/Users/cepe-s3-02/Desktop/David/Bloc 2/notebooks/data/wimbledon2013.csv")

12. Ajouter une colonne `Tournoi` dans les dataframes `rg` et `wb` contenant respectivement les chaînes de caractères `"RG"` et `"WB"`.

In [142]:
rg2013["Tournoi"] = "RG2013"
wb2013["Tournoi"] = "WB2013"


13. Concaténer les deux dataframes dans un nouveau dataframe `tennis`.

In [143]:
majeur2013 = pd.concat( (rg2013, wb2013), ignore_index=True)
print(majeur2013)

                 Player1                  Player2  Round  Result  FNL.1  \
0    Pablo Carreno-Busta            Roger Federer      1       0      0   
1       Somdev Devvarman  Daniel Munoz-De La Nava      1       1      3   
2           Tobias Kamke            Paolo Lorenzi      1       1      3   
3       Julien Benneteau        Ricardas Berankis      1       1      3   
4            Lukas Lacko              Sam Querrey      1       0      0   
..                   ...                      ...    ...     ...    ...   
234             D.Ferrer              J.Del Potro      5       0      0   
235           N.Djokovic                T.Berdych      5       1      3   
236           J.Janowicz                 A.Murray      6       0      1   
237           N.Djokovic              J.Del Potro      6       1      3   
238           N.Djokovic                 A.Murray      7       0      0   

     FNL.2  FSP.1  FSW.1  SSP.1  SSW.1  ...  NPW.2  TPW.2  ST1.2  ST2.2  \
0        3     62     27

14. Utiliser le dataframe `tennis` pour comparer le nombre moyen d'aces par match à chaque niveau du tournoi à Roland Garros et à Wimbledon. Afficher le résultat en format large.

In [144]:
majeur2013["ACE"] = majeur2013["ACE.1"] + majeur2013["ACE.2"]

majeur2013["RoundByT"] = majeur2013["Round"].astype(str) + majeur2013["Tournoi"]

meanAcesByRound2 = majeur2013.groupby("RoundByT")["ACE"].mean()

#print(meanAcesByRound2)

meanAcesByRound3 = majeur2013.groupby(["Round","Tournoi"])["ACE"].mean()

#print(meanAcesByRound3)

format_large = meanAcesByRound3.reset_index().pivot(
    index="Round",
    columns="Tournoi",
    values="ACE"
)

print(format_large)

Tournoi     RG2013     WB2013
Round                        
1        13.476190  21.125000
2        13.193548  23.869565
3        12.562500  24.000000
4         9.125000  24.375000
5         7.000000  26.500000
6        10.000000  27.500000
7         6.000000  13.000000


15. Quelle différence y a-t-il dans le format des noms des joueurs entre les dataframes `rg` et `wb` ?

In [145]:
print(rg2013["Player1"])
print(wb2013["Player1"])


0      Pablo Carreno-Busta
1         Somdev Devvarman
2             Tobias Kamke
3         Julien Benneteau
4              Lukas Lacko
              ...         
120           Rafael Nadal
121         Novak Djokovic
122           David Ferrer
123         Novak Djokovic
124           Rafael Nadal
Name: Player1, Length: 125, dtype: object
0        B.Becker
1          J.Ward
2         N.Mahut
3       T.Robredo
4         R.Haase
          ...    
109      D.Ferrer
110    N.Djokovic
111    J.Janowicz
112    N.Djokovic
113    N.Djokovic
Name: Player1, Length: 114, dtype: object


16. Construire un dataframe `rg_victoires` avec les trois colonnes suivantes pour le tournoi de Roland Garros :
- `joueur` : nom du joueur tel qu'il est donné dans `rg`,
- `nom_joueur` : nom de famille du joueur uniquement,
- `n_victoire` : nombre de matchs gagnés dans le tournoi.

In [ ]:
rg2013["Nom1"] = rg2013["Player1"].str.split(" ", n=1).str[1]
rg2013["Nom2"] = rg2013["Player2"].str.split(" ", n=1).str[1]

wb2013["Nom1"] = wb2013["Player1"].str.split(".", n=1).str[1]
wb2013["Nom2"] = wb2013["Player2"].str.split(".", n=1).str[1]

#print(rg2013["Nom1"])
#print(wb2013["Nom1"])

joueursVic = pd.concat([rg2013["Nom1"], rg2013["Nom2"], wb2013["Nom1"], wb2013["Nom2"]])

joueursVic.drop_duplicates(inplace=True)

#print(joueursVic.head())

def n_victoires(matchs, joueur):

    # Compter les occurrences du joueur dans Player1 et Player2 avec result = 1 ou 0
    count_player1 = matchs[(matchs["Nom1"] == joueur) & (matchs["Result"] == 1)]["Nom1"].count()
    count_player2 = matchs[(matchs["Nom2"] == joueur) & (matchs["Result"] == 0)]["Nom2"].count()
    
    # Calculer le total des matchs joués
    n_victoires = count_player1 + count_player2
    
    #print(joueur + " : " + str(n_victoires))

    return n_victoires

#print(majeur2013)

majeur2013Bis = pd.concat( (rg2013, wb2013), ignore_index=True)

print(n_victoires(majeur2013Bis, "Ferrer"))

for joueur in joueursVic:
    #print(joueur)
    nbVicParJoueur = n_victoires(majeur2013Bis, joueur)
    print(joueur + " : " + str(nbVicParJoueur))
    #joueursVic.loc[playersDf["Joueur"] == joueur, "nVictoires"] = nbVicParJoueur

print(joueursVic.sort_values("nVictoires", ascending=False))


0    Carreno-Busta
1        Devvarman
2            Kamke
3        Benneteau
4            Lacko
dtype: object
9
Carreno-Busta : 0
Devvarman : 1
Kamke : 1
Benneteau : 3
Lacko : 0
Hajek : 1
Mannarino : 2
Simon : 3
Petzschner : 0
Stepanek : 1
Troicki : 5
Monaco : 2
Becker : 0
Bautista Agut : 2
Nieminen : 1
Tsonga : 6
Matosevic : 0
Johnson : 0
Sousa : 1
Granollers : 0
Marchenko : 0
Struff : 2
Darcis : 1
Raonic : 3
Haider-Maurer : 0
Alund : 0
Kavcic : 1
Seppi : 4
Zopp : 0
Sijsling : 3
Gulbis : 3
Berdych : 4
Stakhovsky : 2
Przysiezny : 2
Serra : 0
Mayer : 1
Ramos : 0
De Schepper : 1
Pospisil : 1
Wawrinka : 4
Levine : 1
Zemlja : 3
Kubot : 3
Paire : 4
Beck : 0
Riba : 0
Klizan : 1
Nadal : 7
Verdasco : 4
Delbonis : 1
Youzhny : 5
Berlocq : 0
Kuznetsov : 1
Sock : 1
Haas : 5
Vesely : 0
Bolelli : 0
Tomic : 3
Dolgopolov : 1
Falla : 0
Dodig : 3
Djokovic : 11
Kyrgios : 1
Gimeno-Traver : 1
Montanes : 1
Lopez : 3
Donskoy : 1
Roger-Vasselin : 1
Monfils : 2
Istomin : 1
Haase : 1
Rosol : 1
Harrison : 1
Tursu

C:\Users\cepe-s3-02\AppData\Local\Temp\ipykernel_11348\740011278.py:41: FutureWarning: In a future version of pandas all arguments of Series.sort_values will be keyword-only.
  print(joueursVic.sort_values("nVictoires", ascending=False))


ValueError: No axis named nVictoires for object type Series

17. Construire un dataframe `wb_victoires` avec les trois colonnes suivantes pour le tournoi de Wimbledon :
- `joueur` : nom du joueur tel qu'il est donné dans `wb`,
- `nom_joueur` : nom de famille du joueur uniquement,
- `n_victoire` : nombre de matchs gagnés dans le tournoi.

18. Faire une jointure entre `rg_victoires` et `wb_victoires` sur la colonne `nom_joueur` pour comparer le nombre de victoires par tournoi pour chaque joueur. Expliquer la différence de résultat selon que la jointure est à gauche, à droite, intérieure ou extérieure.